In [3]:
# imports. note you will need to install libraries you do not have
import bokeh
import pandas as pd
import geopandas as gpd

from bokeh.io import output_file, show, curdoc
from bokeh.models.tools import HoverTool
from bokeh.models import (ColumnDataSource, MultiPolygons, Title)
from bokeh.plotting import figure, output_file, output_notebook, show
from shapely.geometry import Polygon

# read data from the file
# data taken from https://simple.wikipedia.org/wiki/List_of_European_countries
input_file = pd.read_csv(r'geodata.csv')
output_file('geodata_output.html')#, mode="inline")

# redundant as we already have the input_file defined but maybe it is easier for
# someone who is starting to understand so I will leave it like this
# as it is easier to just write df instead of input_file
df = pd.DataFrame(input_file)


# first get the data for the world map from geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.to_csv(r'world_data.csv', index = False)
world_map = pd.read_csv(r'world_data.csv')
geom_w = world_map['geometry'] #get the geometry

# here we isolate Europe data
europe = (world.loc[world['continent'] == 'Europe'])
names = [country for country in europe.name]

# here we isolate the names of European countries
countries = []
[countries.append(country) if type(item) == Polygon else [countries.append(country) for i in list(item)] for item, country in zip(europe.geometry, names)]

# here we isolate the polygons (country borders) for Europe
polygons = []
[polygons.append(item) if type(item) == Polygon else [polygons.append(i) for i in list(item)] for item in europe.geometry]

# we define are x and y lists
xs, ys = [], []
xs = [list(polygon.boundary.coords.xy[0]) for polygon in polygons]
ys = [list(polygon.boundary.coords.xy[1]) for polygon in polygons]

############################### BOKEH ###############################

# define some general props of the figure p
p = figure(tools = "pan, wheel_zoom, reset",
           x_range = (-30, 60), y_range = (30, 85), 
           x_axis_location=None, 
           y_axis_location=None, 
           plot_width=800, 
           plot_height=700)

# I do not want a grid so:
p.xgrid.visible = False
p.ygrid.visible = False

# configure plot title, subtitles with respective font attributes
p.title.text = 'Map of Europe with capitals'

p.add_layout(Title(text='Design&Code: Niko Sarcevic (twitter.com/NikoSarcevic)', align='left',
       text_font_size="10px", text_color='#ff6600'), "above")
p.add_layout(Title(text='Data taken from Wikipedia www.simple.wikipedia.org/wiki/List_of_European_countries', align='left',
       text_font_size="7px", text_color='#ff6600'), "below")

p.title.text_color = '#ff6600'
p.title.text_font_size = "20px"

# define the data source for the Europe map layer
source_map = ColumnDataSource(dict(
    xs = xs, 
    ys = ys, 
    countries = countries
))

# define the data source for capitals and some country data (from the df)
source_stats = ColumnDataSource(data=dict(
    lats = df['latitude'],
    longs = df['longitude'],
    capital = df['capital'],
    country = df['country'],
    flags = df['flag'],
    area = df['area'],
    population = df['population'],
    pop_density = df['pop_density'],
    fonts=[
        '<i>italics</i>',
        '<pre>pre</pre>',
        '<b>bold</b>',
        '<small>small</small>',
        '<del>del</del>'
    ]
))

# hover tooltip over any country area
tooltips_countries = """

    <div>
        <span style="float: left; font-size: 12px; color: #696969;">
        <span style="color:#ff6600"> Country: </span> @countries</sub>
    </div>

"""

# hover tooltip over capital cities
tooltips_capitals = """
    <div>
            <img
                src="@flags" height="25" alt="flags" width="50"
                style="float: right; margin: 1px 1px 1px 1px; border-color: #808080"; 
                border="1";
            ></img>
        <span style="left; font-size: 20px; font-weight: bold; color: #696969;">@capital</span>
    </div>
    
    <div>
        <span style="left; font-size: 17px; color: #696969;">@country</span>
    </div>
    <div>
        <span style="float: left; font-size: 15px; color: #696969;">
        <span style="color:#ff6600"> Area: </span> @{area} km<sup>2</sub>
    </div>
    <div>
        <span style="float: left; font-size: 15px; color: #696969;">
        <span style="color:#ff6600"> Population: </span> @{population} </sub>
    </div>
    <div>
        <span style="float: left; font-size: 15px; color: #696969;">
        <span style="color:#ff6600"> Population density: </span> @{pop_density} per km<sup>2</sub>
    </div>
    
"""

# create patches (country shapes) from data extraced from polygons
r1 = p.patches('xs', 'ys', fill_alpha=0.3, fill_color='#D3D3D3', 
          line_color='black', line_width=0.5, source=source_map)

# use a hover over countries
h1 = HoverTool(tooltips=tooltips_countries, renderers=[r1])

# overplot circles that correspond to the lats and longs of European capitals 
r2 = p.circle(x='longs', y='lats', source=source_stats, 
              size=10, color='#ff6600', alpha=0.65)

# use a hover over capital cities
h2 = HoverTool(tooltips=tooltips_capitals, 
               renderers=[r2]) # only hovers the circle

# make sure both hovers are active
p.add_tools(h1, h2)

# show your plot
show(p)
